# Create, modify and work with a general network.

In [1]:
from pymnet import net
import numpy as np
import random
from pymnet import *
import tensorflow as tf
import pandas as pd
import math
import matplotlib.pyplot as plt
import geopandas as gpd

import heapq
import copy
from collections import defaultdict



## Intro

In [7]:
# IMPORTANT: No aspects can be added to an empty network, 
# a 0 aspect network is just a monoplex
monoplex = net.MultilayerNetwork(
    aspects=0,
    noEdge=True,
    directed=False,
    fullyInterconnected=False,
)

monoplex.add_node('a')
monoplex.add_node('b')
monoplex.add_node('c')

monoplex['a','b'] = 1
monoplex['b','c'] = 2
monoplex['c','c'] = 3


In [8]:
multinetwork = net.MultilayerNetwork(
    aspects=1,
    noEdge=True,
    directed=False,
    fullyInterconnected=False,
)

multinetwork.add_layer('l1', 1)
multinetwork.add_layer('l2', 1)

multinetwork.add_node('a', 'l1')
multinetwork.add_node('c', 'l1')
multinetwork.add_node('a', 'l2')
multinetwork.add_node('b', 'l2')

multinetwork['a','b','l1','l2'] = 1
multinetwork['b','c','l2','l1'] = 2
multinetwork['a','a','l1','l2'] = 3
multinetwork['c','a','l1','l1'] = 4


## 1.1. ---Tensorflow functionality---

In [2]:
def __init_from_sparse_tensor__(sparse_tensor,  nodes_names=None, aspects_names=None,directed=False):
    """Initialize a MultilayerNetwork from a sparse tensor.
    Parameters
    ----------
    sparse_tensor : tf.SparseTensor
        The sparse tensor to initialize the network from.  
            Given a sparse tensor of shape (a1,a2,b1,b2,...,an1,an2,n1,n2),
            the first n dimensions are the aspects, and the last two dimensions
            are the nodes. The values of the sparse tensor are the weights of 
    nodes_names : list of str
        The names of the nodes in the network.
    aspects_names : list of list of str
        For each aspect, the names of the layers in the aspect.
    directed : bool
        Whether the network is directed or not. If True, the network is directed.
        If False, the network is undirected. Default is False.
    """

    if sparse_tensor.shape.rank < 2:
        raise ValueError("The sparse tensor must have at least 2 dimensions.")
    
    if len(sparse_tensor.shape[:-2]) % 2 != 0:
        raise ValueError("The sparse tensor must have an even number of dimensions.")
        
    if any([sparse_tensor.shape[i] != sparse_tensor.shape[i+1] for i in range(0, sparse_tensor.shape.rank-2, 2)]):
        raise ValueError("The sparse tensor must have the same size in the first and second dimensions for each aspect.")
    
    if nodes_names and len(nodes_names) != sparse_tensor.shape[-2]:
        raise ValueError("The number of nodes must be equal to the size of the last two dimensions in the sparse tensor.")
    
    if nodes_names is None:
        nodes_names = [f"n{i}" for i in range(sparse_tensor.shape[-2])]

    if aspects_names:
        if len(aspects_names) != sparse_tensor.shape[:-2].rank/2:
            raise ValueError("The number of aspects must be equal to the number of non-nodes dimensions in the sparse tensor: ", sparse_tensor.shape[:-2])
    
        for i in range(len(aspects_names)):
            if len(aspects_names[i]) != sparse_tensor.shape[2*i] or len(aspects_names[i]) != sparse_tensor.shape[2*i+1]:
                raise ValueError(f"The number of layers in aspect {i} must be equal to the size of dimension {i} in the sparse tensor.")
    else:
        aspects_names = [ [f'l{i}_{j}' for j in range(sparse_tensor.shape[2*i])] for i in range(len(sparse_tensor.shape[:-2])//2) ]
    

    # Initialize the network
    n = MultilayerNetwork(
        aspects=len(aspects_names),
        noEdge=False,
        directed=directed,
        fullyInterconnected=True,
    )

    for node in nodes_names:
        n.add_node(node)

    for i in range(len(aspects_names)):
        for j in range(len(aspects_names[i])):
            n.add_layer(aspects_names[i][j], i+1)

    for (index, value) in zip(sparse_tensor.indices, sparse_tensor.values):

        aspects_index = index[:-2]
        if aspects_index is not None:
            edge = [nodes_names[index[-2]],  nodes_names[index[-1]]] + [aspects_names[i//2][aspects_index[i]] for i in range(len(aspects_index))]
        else:
            edge = [nodes_names[index[-2]],  nodes_names[index[-1]]]
        print(edge)
        n[tuple(edge)] = value

    return n
    

In [3]:
def __get_sparse_tensor__(self, return_mappings=False):
    """Get the sparse tensor representation of the network.
    Returns
    -------
    sparse_tensor : tf.SparseTensor
        The sparse tensor representation of the network.
    """
    
    if 'sparse_tensor' in self.__dict__ and self.sparse_tensor is not None:
        return self.sparse_tensor
    # definde dictionary with name and position of each node
    nodes = {}
    for i in range(len(self.slices[0])):
        nodes[list(self.slices[0])[i]] = i

        
    layers = [ {list(self.slices[i])[j] : j for j in range(len(self.slices[i]))} for i in range(1, self.aspects+1)]
    
    indices = []
    values = []
    for edge in self.edges:
        edge_nodes = edge[:2]
        edge_aspects = edge[2:-1]
        
        edge_aspects = [layers[i//2][edge_aspects[i]] for i in range(len(edge_aspects))]
        edge_nodes = [nodes[edge_nodes[i]] for i in range(len(edge_nodes))]

        indices.append(edge_nodes + edge_aspects)
        values.append(edge[-1])

        if not self.directed:
            inverted_edge = edge_nodes[::-1] + edge_aspects[::-1]
            indices.append(inverted_edge)
            values.append(edge[-1])
    
        

    indices = tf.constant(indices, dtype=tf.int64)
    values = tf.constant(values, dtype=tf.float32)
    
    # shape = [len(self.slices[i]) for i in range(self.aspects+1)]
    shape = [len(self.slices[i//2]) for i in range(0,(self.aspects+1)*2)]

    
    sparse_tensor = tf.SparseTensor(indices=indices, values=values, dense_shape=shape)
    sparse_tensor = tf.sparse.reorder(sparse_tensor)  # Ensure the indices are sorted
     
    if return_mappings:
        inv_nodes = {v: k for k, v in nodes.items()}
        inv_layers = [{v: k for k, v in l.items()} for l in layers]
        return sparse_tensor, nodes, layers, inv_nodes, inv_layers
    
    return sparse_tensor


def __get_sparse_tensor_between_layers__(self,l1,l2):
    """
    Get the sparse tensor representation of the network between two layers.
    
    Args:
        l1 (list of str/int) : The names of the first layer.
        l2 (list of str/int) : The names of the second layer. 
        
    Returns:
        tf.sparse.SparseTensor: The sparse tensor representation of the network between the two layers.
    """
    #generate a list fix_index = [l1[0], l2[0], l1[1], l2[1], ..., l1[n-1], l2[n-1]]

    
    sparse, nodes, layers, inv_nodes, inv_layers = __get_sparse_tensor__(self, return_mappings=True)
    for i in range(len(l1)):
        if isinstance(l1[i], str):
            l1[i] = layers[0][l1[i]]
        if isinstance(l2[i], str):
            l2[i] = layers[0][l2[i]]

    fixed_indices = [item for pair in zip(l1, l2) for item in pair]

    # fixed_indices = []
    # for i in range(len(l1)):
    #     fixed_indices.append(l1[i])
    #     fixed_indices.append(l2[i])
    

    return __slice_sparse_tensor__(sparse, fixed_indices)

def __slice_sparse_tensor__(sparse_tensor, fixed_indices):
    """
    Extracts a slice from a 2k-dimensional SparseTensor by fixing the last k dimensions.
    
    Args:
        sparse_tensor (tf.sparse.SparseTensor): The original sparse tensor of shape [d1, d2, ..., d_{2k}]
        fixed_indices (List[int]): A list of length k containing fixed indices for the last k dimensions.
        
    Returns:
        tf.sparse.SparseTensor: A sparse tensor of shape [d1, ..., d_k] corresponding to the slice.
    """
    
    fixed_indices = tf.convert_to_tensor(fixed_indices, dtype=tf.int64)
    num_dims = tf.shape(sparse_tensor.dense_shape)[0]
    k = tf.size(fixed_indices)

    
    # Split indices into first k and last k parts
    first_k = sparse_tensor.indices[:, :num_dims - k]
    last_k = sparse_tensor.indices[:, num_dims - k:]
    
    # Create mask for matching fixed_indices (login and between last_k and fixed_indices)
    mask = tf.reduce_all(tf.equal(last_k, fixed_indices), axis=1)

    
    # Filter indices and values
    new_indices = tf.boolean_mask(first_k, mask)
    new_values = tf.boolean_mask(sparse_tensor.values, mask)
    
    # New shape is the first k dimensions
    new_shape = sparse_tensor.dense_shape[:num_dims - k]
    
    return tf.SparseTensor(indices=new_indices, values=new_values, dense_shape=new_shape)


## 1.2. ---Tensroflow generation ---

In [4]:
def er_general_multilayer(
    n_layers_per_aspect : list,
    n_nodes : int,
    p=0.5,
    randomWeights=False,
    directed=False
):
    """
    Generate a random multilayer network with the given parameters.
    
    Parameters
    ----------
    n_layers_per_aspect : list of int
        The number of layers per aspect. Default is [2,2].
    n_nodes : int
        The number of nodes in the network. Default is 8.
    p : float
        The probability of creating an edge between two nodes. Default is 0.5.
    randomWeights : bool
        Whether to use random weights for the edges or not. Default is False.
    directed : bool
        Whether the network is directed or not. Default is False.
    fullyInterconnected : bool
        Determines if the network is fully interconnected, i.e. all nodes
       are shared between all layers.

    Returns
    -------
    net : MultilayerNetwork
        The generated multilayer network.
    
    """
    mnet = MultilayerNetwork(
        aspects=len(n_layers_per_aspect),
        directed=directed
    )

    for node in range(n_nodes):
        mnet.add_node(node)
    if not n_layers_per_aspect:
        for aspect in range(len(n_layers_per_aspect)):
            for layer in range(n_layers_per_aspect[aspect]):
                mnet.add_layer(layer, aspect+1)
    
    for edge in __generate_random_edges_er(n_nodes, n_layers_per_aspect, p):
        if randomWeights:
            weight = random.random()
        else:
            weight = 1
        mnet[tuple(edge)] = weight

    
    return mnet

def __generate_random_edges_er(n_nodes, aspect_sizes, p, directed=False):
    """
    Efficient generator of random edges in a multilayer graph.

    Each node-layer is a tuple of integers: (node, layer1, ..., layerN)

    Parameters:
        n_nodes (int): Number of nodes.
        aspect_sizes (list of int): Number of layers for each aspect.
        p (float): Probability of including an edge between two node-layers.

    Yields:
        tuple: An edge as a tuple of two node-layer tuples.
    """
    if len(aspect_sizes) > 0:
        total_layers = 1
        for size in aspect_sizes:
            total_layers *= size
        total_node_layers = n_nodes * total_layers

        # Helper: convert flat index to node-layer tuple
        def index_to_node_layer(idx):
            node = idx // total_layers
            rest = idx % total_layers
            layers = []
            for size in reversed(aspect_sizes):
                layers.append(rest % size)
                rest //= size
            return (node, *reversed(layers))

        for i in range(total_node_layers):
            for j in range(i + 1, total_node_layers):
                if random.random() < p:
                    nl1 = index_to_node_layer(i)
                    nl2 = index_to_node_layer(j)
                    yield (nl1[0], nl2[0], *nl1[1:], *nl2[1:])  # Flatten the tuple
    else:
        for i in range(n_nodes):
            for j in range(i + 1, n_nodes):
                if random.random() < p:
                    yield (i, j)

In [5]:
def ws_multiplex(
        n_nodes,
        n_layers,
        degrees = [],
        rewiring_probs = [],
) :
    """Generate multilayer Watts-Strogatz network.

    The produced multilayer network has a single aspect.

    Parameters
    ----------
    n_nodes : int
        Number of nodes
    n_layers : int
        Number of layers
    degrees : list
        List of degrees for each layer
    p_rewiring : list
        List of rewiring probabilities for each layer
    """


    multplex = MultiplexNetwork(couplings='categorical')
    rewires = [] 
    for i in range(n_nodes):
        multplex.add_node(i)

    for i in range(n_layers):
        multplex.add_layer(i)

    for i in range(n_layers):
        deg = degrees[i]//2
        for j in range(n_nodes):
            for edges in range(deg):
                multplex.A[i][j,(j+edges+1)%n_nodes] = 1
                
    for i in range(n_layers):
        for j in range(n_nodes):
                # iterate on neighbors of node j in layer i
                neighbors = list(multplex._iter_neighbors_out((j,i),(None,i)))
                not_neighbors = _get_not_neighbors_layer(multplex, j, i)

                for neighbor in neighbors:
                    if random.random() < rewiring_probs[i] and len(not_neighbors) > 0:
                        rewired = random.choice(not_neighbors)
                        print(((j,i), neighbor), " to ", ((j,i), (rewired,i)))
                        multplex.A[i][j,neighbor[0]] = 0
                        multplex.A[i][j,rewired] = 1
                        rewire = ((j,i),(neighbor,i))
                        rewires.append(rewire)
                        
    
    return multplex, rewires

def _get_not_neighbors_layer(net, node, layer, loops = False):
    nodelayers = [(x,layer)  for x in list(net.iter_nodes(layer))]
    neighbors = list(net._iter_neighbors((node, layer),(None,layer)))
    not_neighbors = [node for node in nodelayers if node not in neighbors]
    if not loops:
        not_neighbors = [n[0] for n in not_neighbors if n != (node,layer)]
    return not_neighbors


In [6]:
def generate_random_multilayer_edges_streaming(n_nodes, aspects, p):
    """
    Generate random edges for a multilayer graph with minimal memory usage.
    Each edge connects two different node-layers with probability p.

    Parameters:
        n_nodes (list): List of node identifiers.
        aspects (list of lists): List of aspects, each a list of possible values.
        p (float): Probability of including an edge.

    Yields:
        tuple: An edge as a tuple of two node-layer tuples.
    """


    total_layers = 1
    for a in aspects:
        total_layers *= len(a)
    total_nodes = len(n_nodes) * total_layers

    # Use indexed access to avoid building the list
    def node_layer_at(index):
        node_index = index // total_layers
        layer_index = index % total_layers
        layer_values = []
        for a in reversed(aspects):
            layer_values.append(a[layer_index % len(a)])
            layer_index //= len(a)
        return (n_nodes[node_index], *reversed(layer_values))

    # Iterate through pairs by index (i < j) to avoid storing all pairs
    for i in range(total_nodes):
        for j in range(i + 1, total_nodes):
            if random.random() < p:
                yield (node_layer_at(i), node_layer_at(j)) 

## 1.3. ---Centrality measures--

####        PCI functions

In [7]:
def dijkstra(network, start, max_distance = 100000):
    """
    Given a multilayer network and a node of the network,
    this function returns a dictionary with the distances from the
    node to all the other nodes in the network.

    Parameters
    ----------
    n : MultilayerNetwork
        The multilayer network to analyze
    start : tuple
        The node to analyze
    max_distance : int
        The maximum distance to calculate

    Returns
    -------
    distances : dict
        A dictionary with the distances from the node to all the other nodes
    """
    prio_queue = [(0,start)]  
    distances = { start : 0}
    visited = set()

    while prio_queue:
        distance, node = heapq.heappop(prio_queue)
        if node in visited:
            continue
        visited.add(node)
        for neighbor in network._iter_neighbors(node):
            if neighbor not in visited and distance+1 <= max_distance and neighbor not in distances:
                heapq.heappush(prio_queue, (distance+1,neighbor))
                distances[neighbor] = distance+ network[network._nodes_to_link(node, neighbor)]
    
    return distances


In [8]:
def layer_connections(net, node):
    """
    Given a multilayer network and a node of the network,
    this function returns a dictionary with the number of connections
    of the node to each layer

    Parameters
    ----------
    n : MultilayerNetwork
        The multilayer network to analyze
    node : tuple
        The node to analyze

    Returns
    -------
    connections : dict
        A dictionary with the number of connections of the node to each layer
    """
    connections = {}
    for neighbor in net._iter_neighbors(node):
        layer = neighbor[1:]
        if layer in connections:
            connections[layer] += 1
        else:
            connections[layer] = 1
    return connections

In [9]:
# definition of the PCI functions


def mu_PCI(net, node, mu = 1):
    """Given a multilayer network and a node of the network,
        this function returns the \mu-PCI of the node. The mu-PCI
        of a node is the maximum number k of nodes that are at most
        mu hops away from the node and have a degree of at least k.

    
        Parameters
        ----------
        net : MultilayerNetwork
            The multilayer network to analyze
        node : tuple
            The node to analyze
        mu : float
            The mu maximum distance to consider

        Returns
        -------
        mu_PCI : int
            The \mu-PCI of the node  

    """
    distances = dijkstra(net, node, mu)
    # remove node from distances
    distances.pop(node, None)
    degrees = np.array([net._get_degree(node) for node in distances])
    # sort the degrees descending
    degrees = np.sort(degrees)[::-1]

    # indices = np.arange(1,len(degrees)+1)
    seq = (k for k, d in enumerate(degrees, start=1) if d >= k)
    if not seq:
        mu_PCI = 0
    else:
        mu_PCI = max(seq, default=0)
    return mu_PCI
        


def mlPCI_n(net,node,n = 1):
    """
    Given a multilayer network and a node of the network,
    this function returns the ml-PCI_n of the node. The ml-PCI_n is the 
    maximum number k of neighbors of the node that are connected to at least
    n different layers with k edges.


    Parameters
    ----------
    net : MultilayerNetwork
        The multilayer network to analyze
    node : tuple
        The node to analyze
    n : int
        The number of layers to consider

    Returns
    -------
    mlPCI_n : int
        The ml-PCI_n of the node    
    """
    #layer_degrees is a numpy array with k rows and n columns, being k the number of neighbors of the node and n the parameter n
    layer_degrees = np.zeros(len([1 for _ in net._iter_neighbors(node)]))
    # for each neighbor of the node, get the number of connections to each layer and store it in layer_degrees
    for i, neighbor in enumerate(net._iter_neighbors(node)):
        layer_conn = layer_connections(net, neighbor)
        layer_conn = sorted(list(layer_conn.values()),reverse=True)
        if len(layer_conn) < n:
            layer_degrees[i] = 0
        else:
            ey = layer_conn[n-1]
            layer_degrees[i] = ey
        
    layer_degrees = sorted(layer_degrees,reverse=True)
    
    # mlPCI_n = np.max(np.where(layer_degrees >= indices, indices, 0))
    seq = (k for k, d in enumerate(layer_degrees, start=1) if d >= k)
    # if seq is empty, return 0
    if not seq:
        mlPCI_n = 0
    else:
    # get the maximum k such that d >= k
        mlPCI_n = max(seq, default=0)


    return mlPCI_n
                    
   
def allPCI(net,node):
    """
    Given a multilayer network and a node of the network,
    this function returns the all-PCI of the node. The all-PCI is the
    maximum number k of neighbors of the node that are connected
    to all the layers of the network.

    Parameters
    ----------
    net : MultilayerNetwork
        The multilayer network to analyze
    node : tuple
        The node to analyze

    Returns
    -------
    all_PCI : int
        The all-PCI of the node
    """
    aspects_lens = np.array([len(net.slices[i]) for i in range(1,net.aspects+1)])
    prod = aspects_lens.prod()

    all_PCI =mlPCI_n(net,node,prod)
    return all_PCI 

def lsPCI(net,node):
    """
    Given a multilayer network and a node of the network,
    this function returns the ls-PCI of the node. The ls-PCI is the
    maximum number k of neighbors of the node that are connected to
    at least k different nodes of at least k different layers.

    Parameters
    ----------
    net : MultilayerNetwork
        The multilayer network to analyze
    node : tuple
        The node to analyze

    Returns
    -------
    ls_PCI : int
        The ls-PCI of the node
    """
    neighbors_layer_degrees = []
    for neighbor in net._iter_neighbors(node):
        layer_conn = layer_connections(net, neighbor)
        neighbors_layer_degrees.append(sorted(list(layer_conn.values()),reverse=True))
    dim2_seq = [len(layer_conn) for layer_conn in neighbors_layer_degrees]
    if len(dim2_seq) == 0:
        return 0
    

    nl_matrix = np.zeros((len(neighbors_layer_degrees),max([len(layer_conn) for layer_conn in neighbors_layer_degrees])))
    for i, layer_conn in enumerate(neighbors_layer_degrees):
        nl_matrix[i,:len(layer_conn)] = layer_conn

    lsPCI = 0
    for i in range(0,nl_matrix.shape[1]):
        discarded = np.where(nl_matrix[:,i] < i)[0]
        if len(discarded) > i:
            lsPCI = i
            break
    return lsPCI
    

#### Eigencentrality

In [10]:
def sparse_power_iteration(sparse_matrix, num_iters=1000):
    n = sparse_matrix.dense_shape[1]
    sparse_matrix = tf.sparse.reorder(sparse_matrix)  # ensure canonical order
    b_k = tf.random.normal([n, 1])  # column vector

    for _ in range(num_iters):

        b_k1 = tf.sparse.sparse_dense_matmul(sparse_matrix, b_k)
        norm = tf.norm(b_k1)
        b_k = b_k1 / norm

    return tf.squeeze(b_k)


In [11]:
def independent_layer_eigenvector_centrality(self):
    # if tyupe of layers is not int, map them to int
    layers_to_int = {layer: i for i, layer in enumerate(self.get_layers())}
    int_to_layers = {i: layer for i, layer in enumerate(self.get_layers())}
    layers = [layers_to_int[layer] for layer in self.get_layers()]
    layers = tf.convert_to_tensor(layers, dtype=tf.int32)
    # layers = tf.convert_to_tensor(list(self.get_layers()), dtype=tf.int32)

    def map_fn_body(li):
        # li is a scalar tensor (layer index)
        layer_matrix = __get_sparse_tensor_between_layers__(self, tf.expand_dims(li, 0), tf.expand_dims(li, 0))
        layer_matrix = tf.sparse.reorder(layer_matrix)
        eigvec = sparse_power_iteration(layer_matrix)  # returns [n,] or [n,1]
        return eigvec

    eigenvectors = tf.map_fn(map_fn_body, layers, fn_output_signature=tf.TensorSpec([None], tf.float32))
    return eigenvectors, int_to_layers


In [12]:
def uniform_eigenvector_centrality(self):
    """
    Compute the eigenvector centrality for each layer in a uniform manner.
    
    This function computes the eigenvector centrality for the aggregated adjacency matrix of all layers in the network.
    It sums the adjacency matrices of all layers and then applies the power iteration method to compute the eigenvector centrality.

    Returns
    -------
    tf.Tensor
        A tensor containing the eigenvector centrality for each layer.
    """
    layers = self.get_layers()

    sum_tensor = None

    for li in layers:
        # Get the sparse tensor for the current layer
        layer_matrix = __get_sparse_tensor_between_layers__(self, [li], [li])
        layer_matrix = tf.sparse.reorder(layer_matrix)  # ensure canonical order
        
        if sum_tensor is None:
            sum_tensor = layer_matrix
        else:
            sum_tensor = tf.sparse.add(sum_tensor, layer_matrix)


    # Compute the eigenvector centrality for the aggregated layer matrix
    eigvec = sparse_power_iteration(sum_tensor)
    return eigvec


In [13]:
def local_heterogeneus_eigenvector_centrality(self:MultilayerNetwork, weights = None ):
    #weights must be a [n_layers,n_layers] dense tensor, where n_layers is the number of layers in the network
    n_layers = len(self.get_layers())
    tensor, nodes_to_int, layers_to_int, int_to_nodes, int_to_layers = __get_sparse_tensor__(self, return_mappings=True)
    
    if (weights is not None) and (weights.shape[0] != n_layers or weights.shape[1] != n_layers):
        raise ValueError(f"Weights must be a square tensor of shape [{n_layers}, {n_layers}]. Got {weights.shape}.")
    
    if weights is None:
        weights = tf.constant(np.identity(n_layers), dtype=tf.float32)

    A_star = __sparse_masked_tensordot(tensor, weights)

    A_star = tf.sparse.reorder(A_star)  # Ensure the sparse tensor is in canonical order
    
    layers_tensor = tf.convert_to_tensor(list(range(n_layers)), dtype=tf.int32)
    # layers = tf.convert_to_tensor(list(self.get_layers()), dtype=tf.int32)

    def compute_eigvec(li):
        layer_matrix = __slice_sparse_tensor__(A_star, [li,li])
        layer_matrix = tf.sparse.reorder(layer_matrix)
        eigvec = sparse_power_iteration(layer_matrix)
        return eigvec
    
    eigenvectors = tf.map_fn(compute_eigvec, layers_tensor, fn_output_signature=tf.TensorSpec([None], tf.float32))
    return eigenvectors, nodes_to_int, layers_to_int, int_to_nodes, int_to_layers




def __sparse_masked_tensordot(A_sparse: tf.SparseTensor, W: tf.Tensor) -> tf.SparseTensor:
    """
    Args:
        A_sparse: tf.SparseTensor of shape [n, n, d, d]
        W: Tensor of shape [d, d], weights for combining diagonals
    
    Returns:
        A_sparse_star: tf.SparseTensor of shape [n, n, d, d], where only entries at [:,:,k,k]
                       are non-zero, and:
            A_star[i,j,k,k] = sum_{t=k}^{d-1} W[k, t] * A[i,j,t,t]
    """
    A_sparse = tf.sparse.reorder(A_sparse)
    indices = A_sparse.indices     # [nnz, 4]
    values = A_sparse.values       # [nnz]
    dense_shape = A_sparse.dense_shape
    n, _, d1, d2 = dense_shape.numpy()
    assert d1 == d2, "Expected square last dimensions for diagonals."

    # Extract only diagonal entries: where indices[:,2] == indices[:,3]
    is_diag = tf.equal(indices[:, 2], indices[:, 3])
    diag_indices = tf.boolean_mask(indices, is_diag)
    diag_values = tf.boolean_mask(values, is_diag)
    diag_t = diag_indices[:, 2]  # the shared diagonal index t

    output_entries = {}  # (i, j, k) -> value

    for k in range(d1):
        # Select entries with t >= k
        mask_k = tf.greater_equal(diag_t, k)
        selected_indices = tf.boolean_mask(diag_indices, mask_k)
        selected_values = tf.boolean_mask(diag_values, mask_k)
        selected_t = tf.boolean_mask(diag_t, mask_k)

        # Apply weights: W[k, t]
        weights = tf.gather(W[k], selected_t)
        weighted_vals = selected_values * weights

        for idx, val in zip(selected_indices.numpy(), weighted_vals.numpy()):
            i, j, t, _ = idx
            key = (i, j, k, k)
            output_entries[key] = output_entries.get(key, 0.0) + val

    final_indices = tf.constant(list(output_entries.keys()), dtype=tf.int64)
    final_values = tf.constant(list(output_entries.values()), dtype=tf.float32)

    return tf.SparseTensor(indices=final_indices, values=final_values, dense_shape=dense_shape)





In [14]:
def khatri_rao_weighted_block_sparse(W: tf.Tensor, A_sparse: tf.SparseTensor) -> tf.SparseTensor:
    """
    Compute the Khatri-Rao block product of a sparse tensor:
        A^⊗[i,j] = W[i,j] * A_j, where A_j = A_sparse[:,:,j,j]

    Args:
        W: Tensor of shape [m, m]
        A_sparse: SparseTensor of shape [n, n, m, m] (assumes A_j = A[:,:,j,j])

    Returns:
        A dense tensor of shape [n*m, n*m]
    """
    n = tf.cast(A_sparse.dense_shape[0], tf.int32)
    m = tf.shape(W)[0]
    
    indices = A_sparse.indices
    values = A_sparse.values

    # Only keep diagonal slices: i == j in the last two dims
    diag_mask = tf.equal(indices[:, 2], indices[:, 3])
    diag_indices = tf.boolean_mask(indices, diag_mask)
    diag_values = tf.boolean_mask(values, diag_mask)

    output_indices = []
    output_values = []

    for i in range(m):
        for j in range(m):
            # Extract entries of A_j = A[:, :, j, j]
            mask_j = tf.equal(diag_indices[:, 2], j)
            indices_j = tf.boolean_mask(diag_indices, mask_j)[:, :2]  # [row, col]
            values_j = tf.boolean_mask(diag_values, mask_j)

            # Compute block offset
            row_offset = i * n
            col_offset = j * n
            
            # Shift indices into block [i,j]
            shifted_indices = indices_j + tf.cast(tf.stack([row_offset, col_offset]), tf.int64)

            # Scale values by W[i, j]
            scaled_values = tf.cast(W[i, j], tf.float32) * values_j

            output_indices.append(shifted_indices)
            output_values.append(scaled_values)

    all_indices = tf.concat(output_indices, axis=0)
    all_values = tf.concat(output_values, axis=0)
    dense_shape = tf.constant(np.array([n*m, n*m]), dtype=tf.int64)

    return tf.SparseTensor(indices=all_indices, values=all_values, dense_shape=dense_shape)


In [15]:
def global_heterogeneus_eigenvector_centrality(self:MultilayerNetwork, weights = None):
    """
    Compute the global eigenvector centrality for a multilayer network with heterogeneous weights.
    
    Parameters
    ----------
    self : MultilayerNetwork
        The multilayer network to compute the eigenvector centrality for.
    weights : tf.Tensor, optional
        A square tensor of shape [n_layers, n_layers] representing the weights for each layer pair.
        If None, an identity matrix is used.

    Returns
    -------
    tf.Tensor
        A tensor containing the global eigenvector centrality for each layer.
    """
    
    n_layers = len(self.get_layers())
    
    if (weights is not None) and (weights.shape[0] != n_layers or weights.shape[1] != n_layers):
        raise ValueError(f"Weights must be a square tensor of shape [{n_layers}, {n_layers}]. Got {weights.shape}.")
    
    if weights is None:
        weights = tf.identity(np.identity(n_layers))

    sparse_tensor, nodes_to_int, layers_to_int, int_to_nodes, int_to_layers = __get_sparse_tensor__(self, return_mappings=True)
    
    A_block = khatri_rao_weighted_block_sparse(weights, sparse_tensor)

    A_block = tf.sparse.reorder(A_block)  # Ensure the sparse tensor is in canonical order
    
    eigvec = sparse_power_iteration(A_block)
    #divide eigvec in n_layers parts of size n_nodes
    n_nodes = len(self.slices[0]) 
    eigvec = tf.reshape(eigvec, (n_layers,n_nodes))
    
    return eigvec, nodes_to_int, layers_to_int, int_to_nodes, int_to_layers

## 1.4. ---Deteccion Comunidades---


In [16]:
multinet_coms = net.MultiplexNetwork(
    couplings='categorical',
)
multinet_coms.add_layer(1)
multinet_coms.add_layer(2)
multinet_coms.add_layer(3)

multinet_coms.add_node(1)
multinet_coms.add_node(2)
multinet_coms.add_node(3)
multinet_coms.add_node(4)
multinet_coms.add_node(5)
multinet_coms.add_node(6)

multinet_coms[(1,1)][(3,1)] = 1
# multinet_coms[(1,1)][(2,1)] = 1
multinet_coms[(1,1)][(4,1)] = 1
multinet_coms[(1,1)][(5,1)] = 1
multinet_coms[(2,1)][(4,1)] = 2
multinet_coms[(4,1)][(6,1)] = 1
multinet_coms[(4,1)][(3,1)] = 1
multinet_coms[(5,1)][(6,1)] = 1


multinet_coms[(2,2)][(3,2)] = 1
multinet_coms[(2,2)][(4,2)] = 2
# multinet_coms[(2,2)][(6,2)] = 1
multinet_coms[(4,2)][(3,2)] = 1
multinet_coms[(4,2)][(1,2)] = 1
multinet_coms[(4,2)][(5,2)] = 1
multinet_coms[(5,2)][(3,2)] = 1
multinet_coms[(5,2)][(6,2)] = 1

# multinet_coms[(1,3)][(2,3)] = 1
multinet_coms[(1,3)][(3,3)] = 1
multinet_coms[(1,3)][(4,3)] = 1
multinet_coms[(2,3)][(3,3)] = 1
multinet_coms[(2,3)][(4,3)] = 1
multinet_coms[(3,3)][(4,3)] = 1
# multinet_coms[(3,3)][(5,3)] = 1
# multinet_coms[(4,3)][(6,3)] = 1
multinet_coms[(5,3)][(6,3)] = 1



#### Louvain

In [17]:
# Mucha et al. (2010)	for new modularity in multiplex networks
def modularity(communities):
    mod = 0
    graph_size = communities.get('graph_size', 0)
    for community in communities['com2nodes'].keys():
        comm_inner = communities['com_inner_weight'][community]
        comm_total = communities['com_total_weight'][community]
        mod += comm_inner/(2*graph_size) - (comm_total/(2*graph_size))**2
    return mod



def merge_coms(coms,com1,com2,weigth_inter_coms):
    new_coms = coms.copy()
    new_coms['com2nodes'][com1] = new_coms['com2nodes'][com1] + new_coms['com2nodes'][com2]

    for node in new_coms['com2nodes'][com2]:
        new_coms['node2com'][node] = com1
        
    # weigth_inter_coms duplicated as those edges become edges between a node and itself  
    new_coms['com_inner_weight'][com1] += new_coms['com_inner_weight'][com2] + 2*weigth_inter_coms
    new_coms['com_total_weight'][com1] += new_coms['com_total_weight'][com2] 

    new_coms['com2nodes'].pop(com2)
    new_coms['com_inner_weight'].pop(com2)
    new_coms['com_total_weight'].pop(com2)

    return new_coms

def weights_inter_coms(net,coms):
    inter_coms = defaultdict(int)
    for edge in net.edges:
        # last value in edge is the weight, so we ignore it
        edge = tuple(list(edge)[:-1])
        n1,n2 = net._link_to_nodes(edge)
        com1 = coms['node2com'][n1]
        com2 = coms['node2com'][n2]
        if com1 != com2:
            inter_coms[(min(com1, com2), max(com1, com2))] += net[n1][n2]
    return inter_coms



def louvain_step(net,communities):
    com_keys = list(communities['com2nodes'].keys())
    n_coms = len(com_keys)
    max_objective_value = communities['objective_function_value']
    weights = weights_inter_coms(net,communities)
    
    max_coms = communities

    for i in range(n_coms):
        for j in range(i+1,n_coms):

            c1, c2 = com_keys[i], com_keys[j]
            if (c1, c2) not in weights and (c2, c1) not in weights:
                continue
            w_inter = weights.get((c1, c2), weights.get((c2, c1), 0))

            temp_coms = {
                'com2nodes': {k: v[:] for k, v in communities['com2nodes'].items()},
                'node2com' : communities['node2com'].copy(),
                'com_inner_weight': communities['com_inner_weight'].copy(),
                'com_total_weight': communities['com_total_weight'].copy(),
                'graph_size': communities['graph_size'],
                'objective_function': communities['objective_function'],
                'objective_function_name': communities['objective_function_name'],
                'objective_function_value': communities['objective_function_value'],
            }

            # print(com_keys[i],com_keys[j])
            merged_coms = merge_coms(temp_coms,com_keys[i],com_keys[j],weights[(com_keys[i],com_keys[j])])
            new_objective_value = merged_coms['objective_function'](merged_coms)
            merged_coms['objective_function_value'] = new_objective_value
            
            
            if new_objective_value > max_objective_value:
                max_objective_value = new_objective_value
                max_coms = merged_coms
    return max_coms


def louvain_algorithm(net, obj_function=modularity, max_iter=100):
    """
    Apply the Louvain algorithm to find communities in a multiplex network.
    
    Parameters
    ----------
    net : MultilayerNetwork
        The multilayer network to analyze.
    obj_function : function, optional
        The objective function to optimize (default is modularity).
    max_iter : int, optional
        The maximum number of iterations to run the algorithm (default is 100).
    
    Returns
    -------
    dict
        A dictionary containing the communities and their properties.
    """
    
    communities = init_multiplex_communities_louvain(net, obj_function)
    states = [communities]
    for _ in range(max_iter):
        new_communities = louvain_step(net, communities)
        if new_communities['objective_function_value'] <= communities['objective_function_value']:
            break
        states.append(new_communities)
        communities = new_communities
    
    return states     

def init_multiplex_communities_louvain(net, obj_function=modularity):
    # all representation of a node is within the same community
    coms = list(net.slices[0])

    communities = {
        'com2nodes' : {com : [nl for nl in net.iter_node_layers() if nl[0] == com] for com in coms},
        'node2com' : {},
        'com_inner_weight' : {com : 0 for com in coms},
        'com_total_weight' : {com : 0 for com in coms},
        'graph_size' : len(net.edges),
        'objective_function' : obj_function,
        'objective_function_name' : obj_function.__name__,
        'objective_function_value' : 0,
    }


    for com, nodes in communities['com2nodes'].items():
        inner_w = 0
        total_w = 0
        for node1 in nodes:
            for node2 in nodes:
                inner_w += net[node1][node2]
            for neighbor in net._iter_neighbors(node1):
                total_w += net[node1][neighbor]
        communities['com_inner_weight'][com] = inner_w
        communities['com_total_weight'][com] = total_w
    
    communities['node2com'] = {node: com for com, nodes in communities['com2nodes'].items() for node in nodes}
    communities['objective_function_value'] = obj_function(communities)
    return communities    

In [ ]:
##### new net generation functions #####

def communities_are_neighbors(net, com1, com2, communities):
    """
    Check if two communities are neighbors in the network.
    
    Parameters
    ----------
    net : MultilayerNetwork
        The multilayer network to analyze.
    com1 : str
        The first community.
    com2 : str
        The second community.
    communities : dict
        A dictionary containing the communities of the network.

    Returns
    -------
    bool
        True if the communities are neighbors, False otherwise.
    """
    
    for node1 in communities['com2nodes'][com1]:
        for node2 in net._iter_neighbors(node1):
            if node2 in communities['com2nodes'][com2]:
                return True
    return False

def get_highest_degree_node(net, communities, com):
    """
    Get the node with the highest degree in a given community.
    
    Parameters
    ----------
    net : MultilayerNetwork
        The multilayer network to analyze.
    communities : dict
        A dictionary containing the communities of the network.
    com : str
        The community to analyze.

    Returns
    -------
    tuple
        The node with the highest degree in the community.
    """

    nodes_no_layers_degree = { n_l[0] : 0 for n_l in communities['com2nodes'][com]}

    for node1 in communities['com2nodes'][com]:
        nodes_no_layers_degree[node1[0]] += net._get_degree(node1)
    # get the node with the highest degree
    highest_degree_node = max(nodes_no_layers_degree, key=nodes_no_layers_degree.get)
    return highest_degree_node
        
    
def generate_louvain_communities_net(net,communities):
    """
    Generate a new network from the communities of a multilayer network.
    
    Parameters
    ----------
    net : MultilayerNetwork
        The multilayer network to analyze.
    communities : dict
        A dictionary containing the communities of the network.

    Returns
    -------
    MultilayerNetwork
        A new multilayer network with the communities as nodes and edges between them.
    """
    
    com_net = MultilayerNetwork(aspects=1, directed=False)
    
    com_2_name = {com:"" for com in communities['com2nodes'].keys()}
    for com, nodes in communities['com2nodes'].items():
        com_2_name[com] = get_highest_degree_node(net, communities, com)


    for com in communities['com2nodes'].keys():
        com_net.add_node(com_2_name[com])

    neighbors = defaultdict(set)
    for com in communities['com2nodes'].keys():
        for com2 in communities['com2nodes'].keys():
            if com != com2 and communities_are_neighbors(net, com, com2, communities):
                com_name = com_2_name[com]
                com2_name = com_2_name[com2]
                neighbors[com_name].add(com2_name)
            if com2 != com and communities_are_neighbors(net, com2, com, communities):
                com_name = com_2_name[com]
                com2_name = com_2_name[com2]
                neighbors[com2_name].add(com_name)

    for com, neighbors_set in neighbors.items():
        for neighbor in neighbors_set:
            com_net[(com,1)][(neighbor,1)] = 1  # or any weight you want, here we use 1

    com_sizes = {com_2_name[com]: len(nodes) for com, nodes in communities['com2nodes'].items()}

    

    return com_net, com_sizes, com_2_name


In [19]:
coms_states = louvain_algorithm(multinet_coms, obj_function=modularity, max_iter=100)
for coms in coms_states:
    print("Objective function value:", coms['objective_function_value'])
    print("Communities:", coms['com2nodes'])

Objective function value: 0.28081717451523547
Communities: {1: [(1, 1), (1, 2), (1, 3)], 2: [(2, 1), (2, 2), (2, 3)], 3: [(3, 1), (3, 2), (3, 3)], 4: [(4, 1), (4, 2), (4, 3)], 5: [(5, 1), (5, 2), (5, 3)], 6: [(6, 1), (6, 2), (6, 3)]}
Objective function value: 0.3268698060941828
Communities: {1: [(1, 1), (1, 2), (1, 3)], 2: [(2, 1), (2, 2), (2, 3), (4, 1), (4, 2), (4, 3)], 3: [(3, 1), (3, 2), (3, 3)], 5: [(5, 1), (5, 2), (5, 3)], 6: [(6, 1), (6, 2), (6, 3)]}
Objective function value: 0.36426592797783935
Communities: {1: [(1, 1), (1, 2), (1, 3)], 2: [(2, 1), (2, 2), (2, 3), (4, 1), (4, 2), (4, 3)], 3: [(3, 1), (3, 2), (3, 3)], 5: [(5, 1), (5, 2), (5, 3), (6, 1), (6, 2), (6, 3)]}


#### Leiden

In [20]:
import queue

def init_multiplex_communities_leiden(net, obj_function=modularity):
    # all representation of a node is within the same community
    coms = list(net.slices[0])

    communities = {
        'com2nodes' : {com : [nl for nl in net.iter_node_layers() if nl[0] == com] for com in coms},
        'node2com' : {},
        'com_inner_weight' : {com : 0 for com in coms},
        'com_total_weight' : {com : 0 for com in coms},
        'graph_size' : len(net.edges),
        'neigh_coms' : {com : set() for com in coms},
        'objective_function' : obj_function,
        'objective_function_name' : obj_function.__name__,
        'objective_function_value' : 0,

    }


    for com, nodes in communities['com2nodes'].items():
        inner_w = 0
        total_w = 0
        for node1 in nodes:
            for node2 in nodes:
                inner_w += net[node1][node2]
            for neighbor in net._iter_neighbors(node1):
                total_w += net[node1][neighbor]
            communities['neigh_coms'][com].update([x[0] for x in net._iter_neighbors(node1) if x[0] != node1[0]])

        communities['com_inner_weight'][com] = inner_w
        communities['com_total_weight'][com] = total_w
    
    communities['node2com'] = {node: com for com, nodes in communities['com2nodes'].items() for node in nodes}
    communities['objective_function_value'] = obj_function(communities)
    return communities
    
def leiden_FastNodeMove(net,communities, gamma):
    q = queue.Queue()
    
    coms = list(communities['com2nodes'].keys())
    old_communities = copy.deepcopy(communities)

    random.shuffle(coms)
    for com in coms:
        q.put(com)
    # print(q.qsize())
    elems_in_q = coms

    merge_history = {com: [com] for com in coms}
    
    while not q.empty():
        com1 = q.get()
        elems_in_q.remove(com1)
        # print("step: ", com1)
        if com1 not in communities['com2nodes']:            
            continue
        max_cpm = 0
        max_com = com1
        for com2 in communities['neigh_coms'][com1]:
            weights_inter_coms = weights_inter_com1_com2(net,communities,com1,com2)
            new_cpm=compute_inc_CPM(communities,com1,com2,weights_inter_coms,gamma)
            # print("  com1: ",com1," com2: ",com2," new_cpm: ",new_cpm)
            if new_cpm > max_cpm:
                max_com = com2
                max_cpm = new_cpm
        if max_cpm > 0: 
            # print("  merging with",max_com)
            # print("  max_cpm ",max_cpm)
            weights_inter_coms = weights_inter_com1_com2(net,communities,com1,max_com)
            old_neigh_com1 = communities['neigh_coms'][com1]
            communities = merge_communities(net,communities,com1,max_com,weights_inter_coms)
            
            #append the merge history of max_com to com1
            merge_history[com1] = merge_history[com1] + merge_history[max_com]
            merge_history.pop(max_com, None)  # remove max_com from history

            

            for com in old_neigh_com1:
                if com not in elems_in_q:
                    q.put(com)   
                    elems_in_q.append(com)

    return old_communities, communities, merge_history

def weights_inter_com1_com2(net,coms,com1,com2):
    weight_inter_coms = 0
    for node1 in coms['com2nodes'][com1]:
        for neighbor in net._iter_neighbors(node1):
            if coms['node2com'][neighbor] == com2:
                weight_inter_coms += net[node1][neighbor]
    return weight_inter_coms

def weights_inter_subcom(net, subcoms, subcoms_merged):
    edge_weights = { subcom: 0 for subcom in subcoms_merged }
    for i in range(len(subcoms_merged)):
        subcom = subcoms_merged[i]
        for j in range(i + 1, len(subcoms_merged)):
            other_subcom = subcoms_merged[j]
            if subcom == other_subcom:
                continue
            weight = weights_inter_com1_com2(net, subcoms, subcom, other_subcom)
            edge_weights[subcom] += weight
            edge_weights[other_subcom] += weight
    return edge_weights
                        
def compute_inc_CPM(communities,com1,com2,weights_inter_com1_com2,gamma):
    com1_inner = communities['com_inner_weight'][com1]
    com1_size = len(communities['com2nodes'][com1])
    com2_inner = communities['com_inner_weight'][com2]
    com2_size = len(communities['com2nodes'][com2])
    dec = (com1_inner - gamma*(com1_size*(com1_size-1)/2)) + (com2_inner - gamma*(com2_size*(com2_size-1)/2))
    inc = (com1_inner + com2_inner + weights_inter_com1_com2) - gamma*(com1_size + com2_size)*(com1_size + com2_size - 1)/2
    return inc - dec

def merge_communities(net,communities,com1,com2,weigth_inter_coms):
    communities = communities.copy()
    # merge both communities into the new one
    communities['com2nodes'][com1] = communities['com2nodes'][com1] + communities['com2nodes'][com2]
    # change all nodes to the new community
    for node in communities['com2nodes'][com2]:
        communities['node2com'][node] = com1
    

    # the new internal weight is the sum of both plus twice the sum between the communities
    communities['com_inner_weight'][com1] += communities['com_inner_weight'][com2] + 2*weigth_inter_coms

    # new neighboors
    communities['neigh_coms'][com1] = communities['neigh_coms'][com1].union(communities['neigh_coms'][com2])
    communities['neigh_coms'][com1].discard(com1)  # remove self-loop if exists
    communities['neigh_coms'][com1].discard(com2)  # remove self-loop if exists

    communities['com_total_weight'][com1] = communities['com_total_weight'][com1] + communities['com_total_weight'][com2] + weigth_inter_coms

    #remove the old community
    communities['com2nodes'].pop(com2)
    communities['com_inner_weight'].pop(com2)
    communities['com_total_weight'].pop(com2)
    communities['neigh_coms'].pop(com2)

    # update the neigh_coms of the neighbors, change com2 for com1 in the neigh_coms for other communities
    for k,v in communities['neigh_coms'].items():
        if com2 in v:
            v.remove(com2)
            v.add(com1)
    # if com1 in communities['neigh_coms'][com1]:
    #     communities['neigh_coms'][com1].remove(com1)

    communities['objective_function_value'] = communities['objective_function'](communities)

    return communities
        
def check_interconected_subcoms(net, parent_coms, subcoms, parent_com, subcoms_in_parent, gamma):
        # print(parent_com, subcoms_in_parent)
        weights_inter_subcoms = weights_inter_subcom(net, subcoms, subcoms_in_parent)
        R = []
        for subcom1 in subcoms_in_parent:

            e_v_S = weights_inter_subcoms[subcom1]
            # norm_v = deg of all nodes in subcom1
            norm_v = len(subcoms['com2nodes'][subcom1])
            norm_S = len(parent_coms['com2nodes'][parent_com])
            # edges(subcom1, com - subcom1) >= gamma * ||subcom1|| * (||com|| - ||subcom1||) 
            if e_v_S >= gamma * norm_v * (norm_S - norm_v):
                R.append(subcom1)
                continue
        random.shuffle(R)
        return R

def refine_partition(net, parent_coms, subcoms, merge_history, gamma):
    for com_merged, subcoms_merged in merge_history.items():
        # print("Refining partition for com_merged:", com_merged, "with subcoms_merged:", subcoms_merged)
        R = check_interconected_subcoms(net, parent_coms, subcoms, com_merged, subcoms_merged, gamma)
        singleton = R.copy()
        # print("R: ", R)
        if len(R) < 2:
            # print("Skipping refinement for com_merged:", com_merged, "as R has less than 2 elements.")
            continue
        for subcom1 in R:
            # check if subcom1 is a singleton using singleton list
            if subcom1 not in singleton:
                continue


            T = check_interconected_subcoms(net, parent_coms, subcoms, com_merged, subcoms_merged, gamma)
            # print("     T: ", T)
            if not T:
                continue
            

            # Compute ΔH for each target subcommunity
            inc_CPMs = []
            for subcom2 in T:
                if subcom1 == subcom2:
                    continue
                weights_com1_com2 = weights_inter_com1_com2(net, subcoms, subcom1, subcom2)
                inc_CPM = compute_inc_CPM(subcoms, subcom1, subcom2,weights_com1_com2, gamma)
                inc_CPMs.append((subcom2, inc_CPM))

                        # Softmax-based random selection
            probs = []
            for (subcom2, inc_CPM) in inc_CPMs:
                # print("         subcom1:", subcom1, "subcom2:", subcom2, "inc_CPM:", inc_CPM)
                if inc_CPM > 0:
                    # print(math.exp(0.5 * inc_CPM))
                    probs.append(math.exp(0.5 * inc_CPM))
                else:
                    probs.append(0.0)

            if sum(probs) == 0:
                continue

            chosen = random.choices([x[0] for x in inc_CPMs], weights=probs, k=1)[0]

            if subcom1 in singleton:
                singleton.remove(subcom1)
            if chosen in singleton:
                singleton.remove(chosen)
            # print("         singleton: ", singleton)

            # print('         merging subcom1:', subcom1, 'with subcom2:', chosen, 'with inc_CPM:', inc_CPMs)
            subcoms = merge_communities(net, subcoms, chosen, subcom1, 
                                             weights_inter_com1_com2(net, subcoms, chosen, subcom1))

            subcoms_merged.remove(subcom1)


    return subcoms


def leiden_algorithm(net, gamma=1.0, max_iterations=100):
    """
    Perform the Leiden algorithm for community detection in a multiplex network.
    
    Parameters
    ----------
    net : MultilayerNetwork
        The multilayer network to analyze.
    gamma : float, optional
        Resolution parameter for the Leiden algorithm. Default is 1.0.
    max_iterations : int, optional
        Maximum number of iterations to run the algorithm. Default is 100.

    Returns
    -------
    dict
        A dictionary containing the final communities and their properties.
    """
    
    communities = init_multiplex_communities_leiden(net)
    states = [communities]
    for _ in range(max_iterations):
        old_communities, communities, merge_history = leiden_FastNodeMove(net, communities, gamma)
        communities = refine_partition(net, communities,old_communities, merge_history, gamma)

        # check if the com2nodes of old_communities and communities are the same
        if old_communities['com2nodes'] == communities['com2nodes']:
            print("Convergence reached.")
            break
        else:
            states.append(communities)


    return states

In [21]:
def generate_leiden_communities_net(net, communities):
    """
    Generate a new network from the communities of a multiplex network using the Leiden algorithm.
    
    Parameters
    ----------
    net : MultilayerNetwork
        The multilayer network to analyze.
    communities : dict
        A dictionary containing the communities of the network.

    Returns
    -------
    MultilayerNetwork
        A new multilayer network with the communities as nodes and edges between them.
    """
    
    com_net = MultilayerNetwork(aspects=1, directed=False)


    
    com_2_name = {com:"" for com in communities['com2nodes'].keys()}
    for com, nodes in communities['com2nodes'].items():
        com_2_name[com] = get_highest_degree_node(net, communities, com)


    for com in communities['com2nodes'].keys():
        com_net.add_node(com_2_name[com])

    neighbors = defaultdict(set)
    for com in communities['com2nodes'].keys():
        for com2 in communities['neigh_coms'][com]:
            if com != com2:
                com_name = com_2_name[com]
                com2_name = com_2_name[com2]
                neighbors[com_name].add(com2_name)
    for com, neighbors_set in neighbors.items():
        for neighbor in neighbors_set:
            com_net[(com,1)][(neighbor,1)] = 1  # or any weight you want, here we use 1

    com_sizes = {com_2_name[com]: len(nodes) for com, nodes in communities['com2nodes'].items()}

    return com_net, com_sizes, com_2_name

In [22]:
coms_states = leiden_algorithm(multinet_coms, gamma=1.0, max_iterations=100)
for coms in coms_states:
    print("Objective function value:", coms['objective_function_value'])
    print("Communities:", coms['com2nodes'])

Convergence reached.
Objective function value: 0.28081717451523547
Communities: {1: [(1, 1), (1, 2), (1, 3)], 2: [(2, 1), (2, 2), (2, 3)], 3: [(3, 1), (3, 2), (3, 3)], 4: [(4, 1), (4, 2), (4, 3)], 5: [(5, 1), (5, 2), (5, 3)], 6: [(6, 1), (6, 2), (6, 3)]}


## 1.5 ---Difusion---

In [23]:
def update_state(nl1_state, nl2_state,gamma_nl1,gamma_nl2,beta_nl1_nl2,beta_nl2_nl1):
    #begin cases
    match (nl1_state,nl2_state):

        case (1,1):
            return (np.random.choice([1,2], p=[1-gamma_nl1,gamma_nl1]),
                    np.random.choice([1,2], p=[1-gamma_nl2,gamma_nl2])) 
        case (0,1):
            return (np.random.choice([0,1], p=[1-beta_nl2_nl1,beta_nl2_nl1]),
                    np.random.choice([1,2], p=[1-gamma_nl2,gamma_nl2]))
        case (1,0):
            return (np.random.choice([1,2], p=[1-gamma_nl1,gamma_nl1]),
                    np.random.choice([0,1], p=[1-beta_nl1_nl2,beta_nl1_nl2]))
        
        case (1,2):
            return (np.random.choice([1,2], p=[1-gamma_nl1,gamma_nl1]),
                    2)
        case (2,1):
            return (2,
                    np.random.choice([1,2], p=[1-gamma_nl2,gamma_nl2]))
        case default:
            return (nl1_state,nl2_state)

def SIR_net_diffusion(self, interlayers_beta , intra_gamma, iterations, initial_state=None):
    """Given a multilayer network and a dictionary with
        the labels of the nodes in the form of a tuple (node, layer),
        this function returns the vector of the labels of the nodes
        after the number of iterations specified, using the SIR model.
    
        Parameters
        ----------
        net : MultilayerNetwork
            The multilayer network to analyze
        intra_beta : float
            The beta parameter of the SIR model for the intralayer edges
        inter_beta : float
            The beta parameter of the SIR model for the interlayer edges
        gamma : float
            The gamma parameter of the SIR model for each layer
        iterations : int
            The number of iterations to perform the diffusion

        Returns
        -------
    """
    
    if not initial_state:
        initial_state = {edge_nodes: np.random.choice([0,1],p=[0.8,0.2]) for edge in self.edges for edge_nodes in self._link_to_nodes(edge[:-1])}

    state_list = [initial_state.copy()]
    state = initial_state

    for i in range(iterations):
        state_changed = False  # Track if any node state changes
        new_state = state.copy()
        for edge in self.edges:
            ((nl1_0,nl1_1, weight),nl2) = self._link_to_nodes(edge)
            nl1 = (nl1_0, nl1_1)
            nl1_state = state[nl1]
            nl2_state = state[nl2]

            gamma_nl1 = intra_gamma[tuple(nl1[1:])]
            gamma_nl2 = intra_gamma[tuple(nl2[1:])]
            beta_nl1_nl2 = interlayers_beta[(nl1[1:], nl2[1:])]
            beta_nl2_nl1 = interlayers_beta[(nl2[1:], nl1[1:])]


            # Scale beta by weight
            beta_nl1_nl2 = min(beta_nl1_nl2 * weight, 1.0)
            beta_nl2_nl1 = min(beta_nl2_nl1 * weight, 1.0)

            updated_nl1, updated_nl2  = update_state(
                nl1_state, nl2_state,
                gamma_nl1, gamma_nl2,
                beta_nl1_nl2, beta_nl2_nl1
            )

            if new_state[nl1] != updated_nl1:
                new_state[nl1] = updated_nl1
                state_changed = True

            if new_state[nl2] != updated_nl2:
                new_state[nl2] = updated_nl2
                state_changed = True

        # Stop if nothing changed
        if not state_changed:
            break


        state = new_state
        state_list.append(state.copy())
        #if all states are 0 or 2, then the epidemic is over
        if all(s in (0, 2) for s in state.values()):
            break
            
    return state_list
            


# Apply to Madrid Transport Data

## 2.1. ---read from csv---

In [27]:
def _nodes_to_link(node1, node2):
    """Return a link when tuple of nodes is given in the graph representing
    the multislice structure. I.e. when given (i,s_1,...,s_d),(j,r_1,...,r_d)
    (i,j,s_1,r_1, ... ,s_d,r_d) is returned.
    """
    assert len(node1) == len(node2)
    l = []
    for i, n1 in enumerate(node1):
        l.append(n1)
        l.append(node2[i])
    return tuple(l)


def read_multiplex_node_layer_edges_from_csv(file_path, weighted=False, directed=False):
    """
    Reads multilayer edges from a CSV in node,layer,node,layer format and returns a pymnet network.

    Parameters
    ----------
    file_path : str
        Path to the CSV file.
    weighted : bool
        If True, assumes the last column is edge weight.
    directed : bool
        If True, creates a directed network.
    couplings : str
        Coupling type for pymnet network (only applies to Multiplex).

    Returns
    -------
    MultilayerNetwork
    """
    # Load data
    df = pd.read_csv(file_path)

    # Handle weights if present
    if weighted:
        weights = df.iloc[:, -1]
        edge_data = df.iloc[:, :-1]
    else:
        weights = pd.Series([1] * len(df))
        edge_data = df

    if edge_data.shape[1] != 4:
        raise ValueError("Expected columns: node1, layer1, node2, layer2 (plus optional weight).")

    # Initialize general multilayer network
    net = MultilayerNetwork(directed=directed, aspects=1)

    # Add edges
    for (_, row), weight in zip(edge_data.iterrows(), weights):
        u = (row[0], row[1])  # (node1, layer1)
        v = (row[2], row[3])  # (node2, layer2)
        net[u][v] = weight

    return net

def read_multiplex_edges_from_csv(file_path, weighted=False, directed=False):
    """
    Reads multilayer edges from a CSV and returns a pymnet network.

    Parameters
    ----------
    file_path : str
        Path to the CSV file.
    weighted : bool
        If True, assumes the last column is edge weight.
    directed : bool
        If True, creates a directed network.

    Returns
    -------
    MultiplexNetwork
    """
    # Load data
    df = pd.read_csv(file_path)

    # Handle weights if present
    if weighted:
        weights = df.iloc[:, -1]
        edge_data = df.iloc[:, :-1]
    else:
        weights = pd.Series([1] * len(df))
        edge_data = df.iloc[:, :]
    # Detect number of layers from column count
    n_cols = edge_data.shape[1]
    if n_cols % 2 != 0:
        raise ValueError("Number of columns must be even (pairs of node-layer).")

    # Initialize network
    net = MultilayerNetwork(directed=directed, aspects=1)
    #min wight > 0
    min_weight = weights[weights > 0].min() if weighted else 1


    # Add edges
    for (_, row), weight in zip(edge_data.iterrows(), weights):
        # net[tuple(row)] =1 

        net[tuple(row)] = min_weight/weight if weight > 0 else 0

    return net

In [ ]:
nodes_path = '../Data/Final_data/madrid_transport_nodes.csv'
shp_nodes_path = '../Data/Final_data/final_nodes.csv'
edges_path = '../Data/Final_data/madrid_transport_edges.csv'
edges_no_weight_path = '../Data/Final_data/madrid_transport_edges_no_weight.csv'
madrid_transport_nodes_shp = pd.read_csv(shp_nodes_path)

node_coords = {
    row['stop']: (row['stop_lon'], row['stop_lat'])  # X = lon, Y = lat
    for _, row in madrid_transport_nodes_shp.iterrows()
}



In [28]:
mnet_weighted = read_multiplex_edges_from_csv(edges_path, weighted=True)
mnet_no_weighted = read_multiplex_edges_from_csv(edges_no_weight_path, weighted=False)

In [ ]:
#plot basic map

important_nodes = [
    'AVENIDA DE AMERICA',
    'USERA',
    'BARRIO DEL PILAR',
    'PUERTA DEL SOL',
    'LA MORALEJA',
    'PUERTA DE TOLEDO',
    'PUERTA DEL SUR']


# Custom labels for selected stops
custom_node_labels = {  node:node for node in important_nodes }
# Assign a unique color to each node (using a color palette or named colors)
custom_node_colors = { node: 'red' for node in important_nodes }


nodeAlphaDict = {
    (node, layer): 0.6 if node in custom_node_labels else 0.4
    for (node, layer) in mnet_no_weighted.iter_node_layers()
}
nodeSizeDict = {
    (node, layer): 0.01 if node in custom_node_labels else 0.005
    for (node, layer) in mnet_no_weighted.iter_node_layers()
}

# nodeSizeRuleDict={"rule": "scaled", "scalecoeff": 0.15},
# Build nodeLabelDict and nodeColorDict for all matching node-layer pairs
nodeLabelDict = {
    (node, layer): custom_node_labels[node]
    for (node, layer) in mnet_no_weighted.iter_node_layers()
    if node in custom_node_labels
}

nodeColorDict = {
    (node, layer): custom_node_colors[node]
    for (node, layer) in mnet_no_weighted.iter_node_layers()
    if node in custom_node_colors
}

# Draw the multilayer network
fig = plt.figure(figsize=(36, 24))
ax_n = fig.add_subplot(111, projection='3d')
draw(
    mnet_no_weighted,
    nodeLabelDict=nodeLabelDict,       # Show only selected labels
    nodeLabelRule={},                  # No automatic labels
    nodeCoords=node_coords,            # Use shapefile-coordinates
    defaultNodeLabel="",               # Hide others
    defaultLayerLabelLoc=(-0.1, 0.1),  # Position layer names
    defaultEdgeAlpha=0.4,              # Edge transparency
    defaultNodeLabelAlpha=1,
    defaultNodeLabelColor='red',
    
    nodeColorDict=nodeColorDict,       # Custom node colors
    nodeSizeDict=nodeSizeDict,         # Custom node sizes
    layergap=1,                        # Layer vertical separation
    ax=ax_n
)

fig.savefig('madrid_transport_network.png', dpi=300, bbox_inches='tight')


In [497]:
# --- First: Plot leiden_net with community size scaling ---
# Scale node sizes using leiden_com_sizes (adjust scale factor as needed)
GAMMA = 0.0005

leiden_states = leiden_algorithm(mnet_weighted, gamma=GAMMA, max_iterations=100)
print(len(leiden_states))
print('communities found by leiden algorithm:', len(leiden_states[-1]['com2nodes']))
leiden_com_net, leiden_com_sizes, leiden_com_2_name = generate_leiden_communities_net(mnet_weighted, leiden_states[-1])


nodeSizeDict_leiden = {
    (node, layer): leiden_com_sizes[node] / 1300 # Tune 0.5 as needed
    for (node, layer) in leiden_com_net.iter_node_layers()
    if node in leiden_com_sizes
}
nodeSizeDict_global = {
    (node, layer): 0.01 for (node, layer) in mnet_weighted.iter_node_layers()
}


node_coords = {
    row['stop']: (row['stop_lon'], row['stop_lat'])  # X = lon, Y = lat
    for _, row in madrid_transport_nodes_shp.iterrows()
}
colors = plt.get_cmap('tab20', len(leiden_com_net.slices[0]))
# map each community to a color
nodeColorDict_leiden = {
    node: colors(i) if node in leiden_com_sizes else 'black'
    for i, node in enumerate(leiden_com_net.slices[0])
    if node in leiden_com_sizes
}

# for each node in the community, assign the color of the community
nodeColorDict_leiden = {
    (node, layer): nodeColorDict_leiden[node]
    for (node, layer) in leiden_com_net.iter_node_layers()
    if node in nodeColorDict_leiden
}
# change GETAFE CENTRAL color to COLOMBIA color and vice versa
nodeColorDict_global = {
    node:  nodeColorDict_leiden[( leiden_com_2_name[com],1)]
    for com, nodes in leiden_states[-1]['com2nodes'].items()
    for node in nodes
}



#### PLOTS ####
fig1 = plt.figure(figsize=(36, 24))
ax1 = fig1.add_subplot(111, projection='3d')

draw(
    leiden_com_net,
    nodeCoords=node_coords,
    nodeColorDict=nodeColorDict_leiden,  # Use custom colors for communities
    defaultNodeLabel="",               # Hide labels
    defaultEdgeAlpha=0.4,
    nodeSizeDict=nodeSizeDict_leiden, # Scaled by community size
    layergap=1,
    ax=ax1
)

custom_node_labels = {  node:node for node in leiden_com_net.slices[0] }


fig1.savefig(f'leiden_communities_scaled_gamma_{GAMMA}.png', dpi=300, bbox_inches='tight')


# --- Second: Plot original net with community nodes in red ---
# Identify nodes in leiden_net
leiden_nodes = set(node for node, _ in leiden_com_net.iter_node_layers())


fig2 = plt.figure(figsize=(36, 24))
ax2 = fig2.add_subplot(111, projection='3d')

draw(
    mnet_weighted,
    nodeCoords=node_coords,
    nodeSizeDict=nodeSizeDict_global,  # Use global node sizes
    nodeColorDict=nodeColorDict_global,
    nodeLabelRule={},                  # No automatic labels
    defaultNodeLabel="",
    defaultEdgeAlpha=0.3,
    layergap=1,
    ax=ax2
)

fig2.savefig(f'original_network_highlighted_leiden_nodes_gamma_{GAMMA}.png', dpi=300, bbox_inches='tight')

Convergence reached.
1
communities found by leiden algorithm: 14


In [509]:
# --- First: Plot louvain_net with community size scaling ---
# Scale node sizes using louvain_com_sizes (adjust scale factor as needed)

louvain_states = louvain_algorithm(mnet_no_weighted)
print(len(louvain_states))
print('communities found by louvain algorithm:', len(louvain_states[-1]['com2nodes']))
louvain_com_net, louvain_com_sizes, louvain_com_2_name = generate_louvain_communities_net(mnet_no_weighted, louvain_states[-1])


nodeSizeDict_louvain = {
    (node, layer): louvain_com_sizes[node] / 1000 # Tune 0.5 as needed
    for (node, layer) in louvain_com_net.iter_node_layers()
    if node in louvain_com_sizes
}
nodeSizeDict_global = {
    (node, layer): 0.01 for (node, layer) in mnet_no_weighted.iter_node_layers()
}


node_coords = {
    row['stop']: (row['stop_lon'], row['stop_lat'])  # X = lon, Y = lat
    for _, row in madrid_transport_nodes_shp.iterrows()
}
colors = plt.get_cmap('tab20', len(louvain_com_net.slices[0]))
# map each community to a color
nodeColorDict_louvain = {
    node: colors(i) if node in louvain_com_sizes else 'black'
    for i, node in enumerate(louvain_com_net.slices[0])
    if node in louvain_com_sizes
}

# for each node in the community, assign the color of the community
nodeColorDict_louvain = {
    (node, layer): nodeColorDict_louvain[node]
    for (node, layer) in louvain_com_net.iter_node_layers()
    if node in nodeColorDict_louvain
}
# change GETAFE CENTRAL color to COLOMBIA color and vice versa
nodeColorDict_global = {
    node:  nodeColorDict_louvain[( louvain_com_2_name[com],1)]
    for com, nodes in louvain_states[-1]['com2nodes'].items()
    for node in nodes
}  

#show only labeles of communities with more than 6 nodes 
nodeLabelDict_louvain = {
    (node, layer): node
    for (node, layer) in louvain_com_net.iter_node_layers()
    if node in louvain_com_sizes and louvain_com_sizes[node] > 9
}


#### PLOTS ####
fig1 = plt.figure(figsize=(36, 24))
ax1 = fig1.add_subplot(111, projection='3d')

draw(
    louvain_com_net,
    nodeCoords=node_coords,
    nodeColorDict=nodeColorDict_louvain,  # Use custom colors for communities
    nodeLabelDict=nodeLabelDict_louvain,  # Show only selected labels
    nodeLabelRule={},                  # No automatic labels
    defaultNodeLabel="",               # Hide labels
    defaultEdgeAlpha=0.4,
    nodeSizeDict=nodeSizeDict_louvain, # Scaled by community size
    layergap=1,
    ax=ax1
)

custom_node_labels = {  node:node for node in louvain_com_net.slices[0] }


fig1.savefig(f'imgs/louvain_communities_scaled.png', dpi=300, bbox_inches='tight')


# --- Second: Plot original net with community nodes in red ---
# Identify nodes in louvain_net
louvain_nodes = set(node for node, _ in louvain_com_net.iter_node_layers())


fig2 = plt.figure(figsize=(36, 24))
ax2 = fig2.add_subplot(111, projection='3d')

draw(
    mnet_no_weighted,
    nodeCoords=node_coords,
    nodeSizeDict=nodeSizeDict_global,  # Use global node sizes
    nodeColorDict=nodeColorDict_global,
    nodeLabelRule={},                  # No automatic labels
    defaultNodeLabel="",
    defaultEdgeAlpha=0.3,
    layergap=1,
    ax=ax2
)

fig2.savefig(f'imgs/original_network_highlighted_louvain_nodes.png', dpi=300, bbox_inches='tight')

101
communities found by louvain algorithm: 142


## 2.2. ---Tensorflow functions to Madrid data---

In [575]:
sparse, nodes, layers, inv_nodes, inv_layers = __get_sparse_tensor__(mnet_weighted, return_mappings=True)
sparse_inter = __get_sparse_tensor_between_layers__(mnet_weighted, ['metro'], ['urban'])
print(sparse)

SparseTensor(indices=tf.Tensor(
[[  0   0   0   2]
 [  0   0   2   0]
 [  0  17   0   0]
 ...
 [241 226   0   0]
 [241 241   0   2]
 [241 241   2   0]], shape=(2488, 4), dtype=int64), values=tf.Tensor([0.12346772 0.12346772 0.02445602 ... 0.32881767 0.09640628 0.09640628], shape=(2488,), dtype=float32), dense_shape=tf.Tensor([242 242   3   3], shape=(4,), dtype=int64))


In [573]:
mnet_weighted.slices[0]

{'ABRANTES',
 'ACACIAS',
 'AEROPUERTO T-4',
 'AEROPUERTO T1-T2-T3',
 'ALAMEDA DE OSUNA',
 'ALCORCON CENTRAL',
 'ALFONSO XIII',
 'ALMENDRALES',
 'ALONSO CANO',
 'ALONSO DE MENDOZA',
 'ALONSO MARTINEZ',
 'ALSACIA',
 'ALTO DE EXTREMADURA',
 'ALTO DEL ARENAL',
 'ALUCHE',
 'ALVARADO',
 'ANTON MARTIN',
 'ANTONIO MACHADO',
 'ARGANDA DEL REY',
 'ARGANZUELA-PLANETARIO',
 'ARGÜELLES',
 'ARROYO CULEBRO',
 'ARROYOFRESNO',
 'ARTILLEROS',
 'ARTURO SORIA',
 'ASCAO',
 'ATOCHA',
 'AVENIDA DE AMERICA',
 'AVENIDA DE GUADALAJARA',
 'AVENIDA DE LA ILUSTRACION',
 'AVENIDA DE LA PAZ',
 'AVIACION ESPAÑOLA',
 'BAMBU',
 'BANCO DE ESPAÑA',
 'BARAJAS',
 'BARRIO DE LA CONCEPCION',
 'BARRIO DEL PILAR',
 'BARRIO DEL PUERTO',
 'BATAN',
 'BAUNATAL',
 'BEGOÑA',
 'BILBAO',
 'BUENOS AIRES',
 'CALLAO',
 'CAMPAMENTO',
 'CANAL',
 'CANILLAS',
 'CANILLEJAS',
 'CARABANCHEL',
 'CARABANCHEL ALTO',
 'CARPETANA',
 'CARTAGENA',
 'CASA DE CAMPO',
 'CASA DEL RELOJ',
 'CHAMARTIN',
 'CHUECA',
 'CIUDAD DE LOS ANGELES',
 'CIUDAD LINEAL',

## 2.3. ---Centrality measures to Madrid data---

In [550]:
print(mu_PCI(mnet_no_weighted, ('AVENIDA DE AMERICA', 'metro'), mu = 1))
print(mu_PCI(mnet_no_weighted, ('AVENIDA DE AMERICA', 'metro'), mu = 2))

# for each node in the mnet_no_weightedwork store the mu_PCI, mlPCI_n, allPCI and lsPCI in a dictionary
nl_PCIs = {}
for node in mnet_no_weighted.iter_node_layers():
    nl_PCIs[node] = {
        'mu_PCI': mu_PCI(mnet_no_weighted, node, mu=1),
        'mlPCI_n': mlPCI_n(mnet_no_weighted, node),
        'allPCI': allPCI(mnet_no_weighted, node),
        'lsPCI': lsPCI(mnet_no_weighted, node)
    }

4
7


In [556]:
# show nodes with lsPCI > 1
high_lsPCI_nodes = {node: pci['lsPCI'] for node, pci in nl_PCIs.items() if pci['lsPCI'] > 1}
print(f"{len(high_lsPCI_nodes)} Nodes with lsPCI > 1:")
# for node, lsPCI in high_lsPCI_nodes.items():
#     print(f"{node}: {lsPCI}")

230 Nodes with lsPCI > 1:


In [557]:
# show nodes with lsPCI > 1
high_allPCI_nodes = {node: pci['allPCI'] for node, pci in nl_PCIs.items() if pci['allPCI'] > 1}
print(f"{len(high_allPCI_nodes)} Nodes with allPCI > 1:")
# for node, lsPCI in high_lsPCI_nodes.items():
#     print(f"{node}: {lsPCI}")

0 Nodes with allPCI > 1:


In [595]:
tensor, nodes2int, layers2int, int2nodes, int2layers = __get_sparse_tensor__(mnet_no_weighted, return_mappings=True)
inv_layers

[{0: 'interurban', 1: 'urban', 2: 'metro'}]

In [651]:
ILEC_madrid, ILEC_int2layers = independent_layer_eigenvector_centrality(mnet_no_weighted)
UEC_madrid = uniform_eigenvector_centrality(mnet_no_weighted)
LHEC_madrid, LHEC_nodes2int, LHEC_layers2int, LHEC_int2nodes, LHEC_int2layers  = local_heterogeneus_eigenvector_centrality(mnet_no_weighted)
GHEC_madrid, GHEC_nodes2int, GHEC_layers2int, GHEC_int2nodes, GHEC_int2layers = global_heterogeneus_eigenvector_centrality(mnet_no_weighted)

print("ILEC_madrid:", ILEC_madrid.shape)
print("UEC_madrid:", UEC_madrid.shape)
print("LHEC_madrid:", LHEC_madrid.shape)
print("GHEC_madrid:", GHEC_madrid.shape)

ILEC_madrid: (3, 242)
UEC_madrid: (242,)
LHEC_madrid: (3, 242)
GHEC_madrid: (3, 242)


In [655]:
NODE = 'AVENIDA DE AMERICA'
NODE = 'PLAZA ELIPTICA'
LAYER = 'urban'
print(GHEC_madrid[GHEC_layers2int[0][LAYER]][GHEC_nodes2int[NODE]])

print(LHEC_madrid[LHEC_layers2int[0][LAYER]][LHEC_nodes2int[NODE]])


tf.Tensor(1.2144022e-21, shape=(), dtype=float32)
tf.Tensor(0.19444656, shape=(), dtype=float32)


## 2.4. --- SIR ---

In [ ]:
interlayers_beta = {
    (('metro',),('metro',)): 65000,
    (('metro',), ('urban',)): 4,
    (('metro',), ('interurban',)): 0.9,

    (('urban',), ('urban',)): 50000,
    (('urban',), ('interurban',)): 10000000,
    (('urban',), ('metro',)):4,

    (('interurban',), ('interurban',)): 50000,
    (('interurban',), ('metro',)): 4,

    (('interurban',), ('urban',)): 10000000
}
intra_gamma = {
    ('metro',): 0.1,
    ('urban',): 0.2,
    ('interurban',): 0.2
}

initial_state = {
    node:  0
    for node in mnet_weighted.iter_node_layers()
}

initial_state[('AVENIDA DE AMERICA', 'metro')] = 1  # Infected node
initial_state[('PLAZA ELIPTICA', 'urban')] = 1  # Infected node

nodeSizeDict = {
    (node, layer): 0.01 
    for (node, layer) in mnet_no_weighted.iter_node_layers()
}

final_states = SIR_net_diffusion( mnet_weighted,
    interlayers_beta=interlayers_beta,
    intra_gamma=intra_gamma,
    iterations= 10,
    initial_state=initial_state
)

for i, state in enumerate(final_states):
# plot the network labeling only the starting infected nodes, all infected nodes are in red
    fig = plt.figure(figsize=(36, 24))
    ax = fig.add_subplot(111, projection='3d')

    nodeColorDict = {
        (node, layer): 'red' if state[(node, layer)] == 1 else 'blue' if state[(node, layer)] == 2 else 'gray'
        for (node, layer) in mnet_weighted.iter_node_layers()
    }
    #initial_state eq 1
    nodeLabelDict = {
        (node, layer): node if initial_state[(node, layer)] == 1 else ''
        for (node, layer) in mnet_weighted.iter_node_layers()
    }
    draw(
        mnet_weighted,
        nodeCoords=node_coords,
        nodeColorDict=nodeColorDict,
        nodeLabelDict=nodeLabelDict,       # Show only selected labels
        nodeLabelRule={},                  # No automatic labels
        defaultNodeLabel="",               # Hide labels
        defaultEdgeAlpha=0.4,
        nodeSizeDict=nodeSizeDict, # Scaled by community size
        layergap=1,
        ax=ax
    )
    # print('infected nodes:', [node for node, state in state.items() if state == 1])
    # print('recovered nodes:', [node for node, state in state.items() if state == 2])

    plt.title(f"State after {i} iterations")
    plt.savefig(f'imgs/SIR/sir_state_{i}.png', dpi=300, bbox_inches='tight')
